In [1]:
import requests
import pandas as pd

url = "https://www.land.mlit.go.jp/webland/api/TradeListSearch"
from_date = "20211"
to_date = "20224"
target_area = "14"

parameters = {
    "from": from_date,
    "to": to_date,
    "area": target_area
}

response = requests.get(url, params=parameters)

if response.status_code == 200:
    data = response.json()  # JSON形式のデータを取得
    df = pd.DataFrame(data['data'])
else:
    print("Request was not successful. Status code:", response.status_code)


In [2]:
def RemoveM2(df_, needRemoveVal:list):
    # 指定列のデータからtarget_wordを除去
    # 新規列を追加し、除去した行はフラグを立てる(基本XX以上を想定して、overという列を新規追加)
    for col, target_word, nwe_col_word in needRemoveVal:
        new_col = f'{col}_{nwe_col_word}_flag'
        target_row = df_.loc[:, col].str.contains(target_word).fillna(False)
        print(col, target_word, sum(target_row))
        df_[new_col] = 0
        df_.loc[target_row, new_col] = 1
        df_[col] = df_.loc[:,col].str.replace(target_word,'')
        
    return df_

def changeDataType(df_,changeDataTypeVal:dict):
    # float及びintへの変換
    for col,new_data_type in changeDataTypeVal.items():
        print(col,new_data_type)
        df_[col] = df_.loc[:,col].astype(new_data_type)
    return df_

def extractRow(df_, extractCondionDic):
    # 特定の行の除去
    for col, targetValue in extractCondionDic.items():
        if targetValue =='Nan': # Nanと他のも選びたい場合汎用性ないけど、とりあえず
            df_ = df_[df_[col].isna()]
        else:
            target_row = df_[col].isin(targetValue)
            df_ = df_[target_row]
    return df_

def changeCalendarBuildingYear(df_):
    # 和暦を西暦にする
    
    df_dummy = df_.loc[:,['BuildingYear']].copy()
    df_dummy['Koyomi'] = df_.BuildingYear.str[:2]
    
    df_dummy['KoyomiYear'] = df_dummy.BuildingYear.str[2:].str.replace("年", "")
    df_dummy.loc[df_dummy.KoyomiYear=="", 'KoyomiYear'] = "1900" #上記では戦前が消えてるので
    df_dummy['KoyomiYear'] = df_dummy.KoyomiYear.astype('float')
    
    ### これで、暦と、その年を分けれたはずなので、あとはそれを西暦に直す
    # 昭和+1925,平成+1988, 令和+2018
    df_['BuildingYearW'] = df_dummy['KoyomiYear']
    df_.loc[df_dummy.Koyomi=="戦前", 'BuildingYearW'] = 1900
    df_.loc[df_dummy.Koyomi=="昭和", 'BuildingYearW'] = df_dummy.loc[df_dummy.Koyomi=="昭和", 'KoyomiYear'] + 1925
    df_.loc[df_dummy.Koyomi=="平成", 'BuildingYearW'] = df_dummy.loc[df_dummy.Koyomi=="平成", 'KoyomiYear'] + 1988
    df_.loc[df_dummy.Koyomi=="令和", 'BuildingYearW'] = df_dummy.loc[df_dummy.Koyomi=="令和", 'KoyomiYear'] + 2018
    
    return df_


def spritPeriodTradeYearQuarter(df_):
    # Periodの書式：2022年第２四半期
    df_['TradeYear'] = df_.Period.str[:4].astype('int')
    df_['TradeQuarter'] = df_.Period.str[6].astype('int')
    
    return df_ 

def makeYearOldatTrade(df_):
    df_['AgeAtTrade'] = df_.TradeYear - df_.BuildingYearW
    return df_
    

In [3]:
drop_cols = ['PricePerUnit', 
            #  'Purpose', 
             'Direction', 'Classification']

# val: m2が含まれる値を、何に置換するか
needRemoveVal = [
    ['Frontage','m以上','over'], 
    ['Area','㎡以上','over'],
    ['TotalFloorArea', '㎡以上','over'],
    ['TotalFloorArea', 'm^2未満','under']
]
changeDataTypeVal = {
    'TradePrice':'int',
    'Frontage':'float',
    'Area':'int',
    'UnitPrice':'float',
    'TotalFloorArea':'float',
    'Breadth':'float',
    'CoverageRatio':'float',
    'FloorAreaRatio':'float'
}
extractCondionDic = {
    'Type':['宅地(土地と建物)', '中古マンション等', '宅地(土地)'],
    'Remarks': 'Nan'
    }

df2 = df.drop(drop_cols, axis=1)
df2 = RemoveM2(df2, needRemoveVal)
df2 = changeDataType(df2, changeDataTypeVal)
print(df2.shape)
df2 = extractRow(df2, extractCondionDic)
print(df2.shape)
df2 = changeCalendarBuildingYear(df2)
df2 = spritPeriodTradeYearQuarter(df2)
df2 = makeYearOldatTrade(df2)

Frontage m以上 213
Area ㎡以上 208
TotalFloorArea ㎡以上 81
TotalFloorArea m^2未満 0
TradePrice int
Frontage float
Area int
UnitPrice float
TotalFloorArea float
Breadth float
CoverageRatio float
FloorAreaRatio float
(42953, 28)
(41056, 28)


In [4]:
df2.head()

,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,TradePrice,Area,UnitPrice,LandShape,...,Use,Remarks,Frontage_over_flag,Area_over_flag,TotalFloorArea_over_flag,TotalFloorArea_under_flag,BuildingYearW,TradeYear,TradeQuarter,AgeAtTrade
0,宅地(土地),住宅地,14101,神奈川県,横浜市鶴見区,朝日町,22000000,130,170000.0,不整形,...,NaN,NaN,0,0,0,0,NaN,2021,4,NaN
1,宅地(土地と建物),住宅地,14101,神奈川県,横浜市鶴見区,朝日町,40000000,60,NaN,長方形,...,NaN,NaN,0,0,0,0,2017.0,2022,2,5.0
2,中古マンション等,NaN,14101,神奈川県,横浜市鶴見区,朝日町,22000000,70,NaN,NaN,...,NaN,NaN,0,0,0,0,1983.0,2021,4,38.0
3,中古マンション等,NaN,14101,神奈川県,横浜市鶴見区,朝日町,21000000,70,NaN,NaN,...,NaN,NaN,0,0,0,0,1983.0,2021,4,38.0
4,中古マンション等,NaN,14101,神奈川県,横浜市鶴見区,朝日町,13000000,70,NaN,NaN,...,住宅,NaN,0,0,0,0,1983.0,2021,2,38.0


In [5]:
from_date = "20211"
to_date = "20224"
target_area = "14"
file_name = f"real_estate_data_{from_date}_{to_date}_{target_area}.csv"
df2.to_csv('../datas/'+file_name, index=False)

In [288]:
file_name = f"RealEstateData_{parameters['from']}_{parameters['to']}_{parameters['area']}.csv"
file_name

'RealEstateData_20221_20224_14.csv'

In [76]:
target_col = 'TotalFloorArea'
i = 0
try:
    df.loc[:,target_col].astype('int')
    print('intOK')
    i = 1
except:
    print('int不可')
    
if i == 0:
    try:
        df.loc[:,target_col].astype('float')
        print('floatOK')
    except:
        print('float不可')
df.loc[:,target_col].unique()

int不可
float不可


array(['300', nan, '1700', '1800', '470', '990', '125', '230', '95',
       '600', '2000㎡以上', '240', '580', '120', '190', '370', '260', '130',
       '550', '185', '195', '90', '140', '80', '170', '290', '250', '155',
       '105', '730', '610', '280', '145', '210', '1100', '980', '115',
       '560', '320', '1000', '180', '830', '65', '680', '640', '840',
       '490', '420', '380', '740', '430', '220', '390', '1500', '45',
       '135', '630', '450', '920', '110', '1300', '710', '500', '85',
       '650', '340', '1200', '900', '530', '160', '175', '200', '810',
       '150', '100', '690', '1900', '660', '780', '930', '870', '410',
       '165', '440', '890', '330', '700', '520', '570', '720', '460',
       '350', '1400', '75', '70', '55', '360', '310', '880', '820', '670',
       '480', '400', '910', '590', '760', '750', '950', '60', '960',
       '790', '270', '850', '510', '540', '620', '25', '770', '20', '860',
       '35', '1600', '30', '40', '800', '50', '10', '940'], dtype=obje

In [286]:
df2.pivot_table(index='AgeAtTrade', values='Type',aggfunc='count').sort_values('Type', ascending=False)

,Type
AgeAtTrade,
0.0,2507
1.0,412
31.0,386
34.0,354
30.0,326
...,...
65.0,4
66.0,4
69.0,2


In [257]:
df2.BuildingYear.isna().sum()

4566